In [1]:
#https://keras.io/examples/generative/vae/
#https://keras.io/examples/generative/molecule_generation/

In [2]:
import os
from sklearn.preprocessing import MinMaxScaler
import joblib
import pandas as pd
import h5py
#================== initialization ==================
LATENT_DIM = 128
VAE_LR = 5e-4
PARQUET_PATH = './data/OptionsEOD_STG.parquet'
SCALER_PATH = './data/scaler.gz'
UNIQUE_KEYS = ['QUOTE_DATE','SYMBOL','EXPIRE_DATE']
SCALER_COL  = ['DTE','INTRINSIC_VALUE', 'TOTAL_VOLUME',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK',	'P_VOLUME' ]
H5_PATH = './data/OptTrainData.h5'

In [133]:
#Example
import random
from IPython.display import clear_output,display, HTML
import numpy as np
#load scaler
scaler = MinMaxScaler()
PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_PATH) if 'PartitionDate' in d]
random.shuffle(PartitionDate)
scaler = joblib.load(SCALER_PATH)

DATA  = np.empty((0,) + (20,9) )
for i,partdate in enumerate(PartitionDate) :
    df = pd.read_parquet(PARQUET_PATH,engine='pyarrow'
                                 , filters=[('PartitionDate', '=', partdate)]
                                )
    for opt_id in np.unique( df[["OPTIONS_ID"]].values)[:5]:
        df_filter  = df[df["OPTIONS_ID"]==opt_id]
        DATA = np.vstack((DATA ,[scaler.transform(df_filter[SCALER_COL])]))
        
    break
        
#df_filter[SCALER_COL]


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 20 and the array at index 1 has size 11

In [ ]:
# Save the NumPy array to an HDF5 file
with h5py.File(H5_PATH, 'w') as f:
    dset = f.create_dataset('dataset', data=DATA, chunks=True, compression='gzip')

In [127]:
scaler.transform(df_filter[SCALER_COL]).shape

(20, 9)

In [128]:
DATA = np.empty((0,) + (20,9) )

In [129]:
DATA.shape

(0, 20, 9)

In [131]:
DATA = np.vstack((DATA ,[scaler.transform(df_filter[SCALER_COL])]))

In [110]:
c.shape

(2, 2, 3)

In [135]:
scaler.transform(df_filter[SCALER_COL]).shape

(11, 9)

In [139]:
opt_id

'002818f8ffcd474a16296b681f9e3061'

In [140]:
np.unique( df[["OPTIONS_ID"]].values)[:5]

array(['0003c5884bb909aedb05bae6af9374e2',
       '002818f8ffcd474a16296b681f9e3061',
       '002bc7cf0ea4d94e6f6657963542cd83',
       '0043d9dd7902cca8fd32a4bbc328c305',
       '0050ed7f4178dd6c1beb3645a446cdc5'], dtype=object)

In [146]:
len( df[df["OPTIONS_ID"]=='002818f8ffcd474a16296b681f9e3061'] )

11

In [137]:
df_filter[SCALER_COL]

,DTE,INTRINSIC_VALUE,TOTAL_VOLUME,C_BID,C_ASK,C_VOLUME,P_BID,P_ASK,P_VOLUME
12815,0.0,0.00,447.0,0.00,0.00,0.0,0.00,0.00,0.0
12816,0.0,0.00,447.0,0.00,0.00,0.0,0.00,0.00,0.0
12817,0.0,0.00,447.0,0.00,0.00,0.0,0.00,0.00,0.0
12818,0.0,0.00,447.0,0.00,0.00,0.0,0.00,0.00,0.0
12819,0.0,0.00,447.0,0.00,0.00,0.0,0.00,0.00,0.0
12820,17.0,60.55,447.0,61.04,61.37,0.0,0.05,0.06,6.0
12821,0.0,0.00,447.0,0.00,0.00,0.0,0.00,0.00,0.0
12822,0.0,0.00,447.0,0.00,0.00,0.0,0.00,0.00,0.0
12823,0.0,0.00,447.0,0.00,0.00,0.0,0.00,0.00,0.0
12824,0.0,0.00,447.0,0.00,0.00,0.0,0.00,0.00,0.0


In [6]:
#====================================================================
# gconv_units  last is latent_dim

In [7]:
from src.model import OptionChainGenerator
from src.layer import encoder, decoder

model = OptionChainGenerator(
    encoder(latent_dim = LATENT_DIM, 
            input_shape= (20,3), 
            gconv_units = [3], 
            dense_units = [256, 128], 
            dropout_rate= 0.1
           ), 
    decoder(latent_dim  = LATENT_DIM , 
            output_shape= 1,
            dense_units = [128, 256],
            dropout_rate= 0.1
           )
)
model.summery()
vae_optimizer = tf.keras.optimizers.Adam(learning_rate=VAE_LR)
model.compile(vae_optimizer)

2024-06-22 14:00:04.050751: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[<tf.Tensor 'Placeholder:0' shape=(None, 20, 3) dtype=float32>]
<KerasVariable shape=(3, 3), dtype=float32, path=relational_graph_conv_layer/W>


ValueError: Input 0 of layer "global_average_pooling1d" is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: (1, None, 20, 3)

In [ ]:
#================== train model ==================
history = model.fit([feature_tensor], epochs=EPOCHS)


In [ ]:
import numpy as np

def create_adjacency_matrix(options, threshold=25):
    """
    สร้าง adjacency matrix สำหรับ Options Chain โดยใช้ความใกล้เคียงของ strike price และ DTE
    """
    num_options = len(options)
    adjacency_matrix = np.zeros((num_options, num_options))

    for i in range(num_options):
        for j in range(i, num_options):
            # พิจารณาเชื่อมโยงระหว่าง options หาก strike price ต่างกันไม่เกิน threshold และ DTE เท่ากัน
            if abs(options[i]['strike_price'] - options[j]['strike_price']) <= threshold and options[i]['dte'] == options[j]['dte']:
                adjacency_matrix[i, j] = 1
                adjacency_matrix[j, i] = 1
    
    return adjacency_matrix

options = [
    {'strike_price': 100, 'dte': 30},
    {'strike_price': 105, 'dte': 30},
    {'strike_price': 110, 'dte': 30},
    {'strike_price': 115, 'dte': 30},
    {'strike_price': 120, 'dte': 30},
    {'strike_price': 0, 'dte': 0},  # เปลี่ยนค่าเป็น 0
]

options = [
    {'strike_price': 0, 'dte': 0},
    {'strike_price': 120, 'dte': 30},
    {'strike_price': 150, 'dte': 30},
    {'strike_price': 110, 'dte': 30},
    {'strike_price': 105, 'dte': 30},
    {'strike_price': 100, 'dte': 30},  # เปลี่ยนค่าเป็น 0
]


# ปรับ threshold
threshold = 5

adjacency_matrix = create_adjacency_matrix(options, threshold)
print(adjacency_matrix)

In [ ]:
import numpy as np
A = np.array(
    [[1,2,3],
    [4,5,6],
    [7,8,9]]
    
)

In [ ]:
A[:,:,0]

In [ ]:
def get_variable_name(variable):
    for name, value in globals().items():
        if value is variable:
            return name
    return None

# Example usage
leo = ['a', 'b']
var_name = get_variable_name(leo)
print(var_name)  # Ou